<h1>SI LABOLATORIUM - LISTA 1<h1>

In [2]:
from test_main import *
from timer import Timer
from typing import Callable


pl_grunwaldzki =  Station('PL. GRUNWALDZKI', 51.11145165, 17.06052919)
kwiska = Station('Kwiska', 51.12527525, 16.98342767)
pl_danilowskiego = Station('pl. Daniłowskiego', 51.13845372, 17.05991231)
koszarowa_uniwersytet = Station('KOSZAROWA (Uniwersytet)', 51.14049516, 17.06847989)
grabiszynska_cmentarz = Station('GRABISZYŃSKA (Cmentarz)', 51.08990377, 16.97664203)
fat = Station('FAT', 51.09412632, 16.9783528)
wiejska = Station('Wiejska', 51.07700276, 16.96415159)
poczta_glowna = Station('Poczta Główna', 51.1078565, 17.04563742)
krzyki = Station('KRZYKI', 51.07488366, 17.00656861)

Connection graph loading time: 3.8948 s
Connection graph building time: 1.5814 s
----------------------------------------



<h3>Sąsiednie przystanki dla placu Grunwaldzkiego<h3>

In [3]:
for neighbor in data.graph_dict[pl_grunwaldzki]:
    print(neighbor)

most Grunwaldzki
Kochanowskiego
Kliniki - Politechnika Wrocławska
Piastowska
Reja
Bujwida
PL. GRUNWALDZKI


<h3>10 losowych krawędzi 'wychodzących' z przystanku pl. Grunwaldzki<h3>

In [4]:
print(f"Stacja początkowa".ljust(40) + " | " + "Stacja końcowa".ljust(40) + " | " + "Linia" + " | " + "Czasy przejazdu")
print("-" * 120)
for edge in data.get_all_edges(pl_grunwaldzki)[:10]:
    print(edge)

Stacja początkowa                        | Stacja końcowa                           | Linia | Czasy przejazdu
------------------------------------------------------------------------------------------------------------------------
PL. GRUNWALDZKI                          | most Grunwaldzki                         | 4     | 17:40:00 - 17:41:00
PL. GRUNWALDZKI                          | most Grunwaldzki                         | 4     | 07:52:00 - 07:53:00
PL. GRUNWALDZKI                          | most Grunwaldzki                         | 146   | 12:37:00 - 12:39:00
PL. GRUNWALDZKI                          | most Grunwaldzki                         | 145   | 07:09:00 - 07:11:00
PL. GRUNWALDZKI                          | most Grunwaldzki                         | 16    | 14:32:00 - 14:33:00
PL. GRUNWALDZKI                          | most Grunwaldzki                         | 12    | 13:56:00 - 13:57:00
PL. GRUNWALDZKI                          | most Grunwaldzki                         |

<h3>Kod dla algorytmu Dijkstry<h3>


In [5]:
def dijkstra_with_time(graph, start_station, goal_station, start_time):
    costs = {node: float('inf') for node in graph.graph_dict}
    edges = {node: None for node in graph.graph_dict}
    costs[start_station] = 0
    pq = [(0, start_station, convert_hour(start_time))]
    prev_nodes = {node: None for node in graph.graph_dict}
    while pq:
        current_cost, current_station, current_time = heapq.heappop(pq)
        
        if current_cost > costs[current_station]:
            continue
        if not current_station in graph.graph_dict:
            continue

        for edge in graph.get_all_edges(current_station):
            neighbor = edge.end_station
            current_line = None if edges[current_station] is None else edges[current_station].line
            new_cost = current_cost + get_time_cost(current_time, current_line, edge)

            if new_cost < costs[neighbor]:
                costs[neighbor] = new_cost
                edges[neighbor] = edge
                prev_nodes[neighbor] = current_station
                heapq.heappush(pq, (new_cost, neighbor, edge.end_time))

    return costs[goal_station], create_path(goal_station, edges, prev_nodes)


<h3>Kod dla algorytmu A* dla kryterium czasowego<h3>

In [6]:
def astar_time(graph, start_station, goal_station, start_time, heuristic_fn):
    costs = {node: float('inf') for node in graph.graph_dict}
    edges = {node: None for node in graph.graph_dict}
    costs[start_station] = 0
    pq = [(0, start_station)]
    prev_nodes = {node: None for node in graph.graph_dict}
    while pq:
        _, current_station = heapq.heappop(pq)
        
        if current_station == goal_station:
            break

        if not current_station in graph.graph_dict:
            continue
        
        current_time = convert_hour(start_time) if edges[current_station] is None else edges[current_station].end_time
        current_line = None if edges[current_station] is None else edges[current_station].line

        for edge in graph.get_all_edges(current_station):
            
            neighbor = edge.end_station
        
            new_cost = costs[current_station] + get_time_cost(current_time, current_line, edge)

            if new_cost < costs[neighbor]:
                costs[neighbor] = new_cost
                edges[neighbor] = edge
                prev_nodes[neighbor] = current_station
                new_cost = new_cost + distance_heuristic_fn(
                    goal_station,
                    neighbor,
                    heuristic_fn
                )
                heapq.heappush(pq, (new_cost, neighbor))

    the_best_cost = costs[goal_station]

    # Searching for the nearest stop to an unavailable stop
    if goal_station not in edges:
        min_distances = get_sorted_min_distances(goal_station, edges)
        for station, distance in min_distances.items():
            if station in prev_nodes:
                edges[goal_station] = prepare_P_line_edge(goal_station, edges[station], distance)
                prev_nodes[goal_station] = station
                the_best_cost = convert_hour(start_time) - edges[goal_station].end_time
                break


    return the_best_cost, create_path(goal)

<h3>Kod dla algorytmu A* dla kryterium przesiadek<h3>

In [7]:

def astar_transfers(graph, start_station, goal_station, start_time, heuristic_fn):

    costs = {start_station: 0}
    edges = {node: None for node in graph.graph_dict}
    pq = [(0, start_station, convert_hour(start_time))]
    prev_nodes = {node: None for node in graph.graph_dict}
    while pq:
        _, current_station, current_time = heapq.heappop(pq)
        
        if current_station == goal_station:
            break

        if not current_station in graph.graph_dict:
            continue
      
        current_line = None if edges[current_station] is None else edges[current_station].line

        for edge in graph.get_all_edges(current_station):
            neighbor = edge.end_station

            cost = get_transfer_cost(edge, current_line, current_time, goal_station, graph)
            new_cost = costs[current_station] + cost

            if neighbor not in costs or new_cost < costs[neighbor]:

                costs[neighbor] = new_cost
                edges[neighbor] = edge
                prev_nodes[neighbor] = current_station
                priority = new_cost + transfer_heuristic_fn(
                    neighbor,
                    goal_station,
                    cost > 0,
                    graph,
                    current_time,
                    heuristic_fn,
                    new_cost,
                    edge
                )
                
                heapq.heappush(pq, (priority, neighbor, edge.end_time))

    return costs[goal_station], create_path(goal_station, edges, prev_nodes)



<h3>Heurystyka dla kryterium czasowego<h3>

In [8]:

def distance_heuristic_fn(
        a: Station,
        b: Station,
        distance_fn: Callable[[Station, Station], float]) -> float:
    return distance_fn(a, b) * DEGREES_TO_KILOMETERS_CONVERTER / AVERAGE_VELOCITY_IN_WROCLAW # t = s/v

`distance_fn` to funkcja obliczająca odległość pomiędzy dwoma podanymi przystankami - zwraca wynik liczbowy.

<h3>Heurystyka dla kryterium przesiadkowego<h3>

In [9]:
def transfer_heuristic_fn(
        curr_station: Station,
        goal_station: Station,
        has_changed_line: bool,
        graph: Graph,
        time: int,
        distance_fn: Callable[[Station, Station], float],
        number_of_changes: int,
        curr_edge: Edge
    ) -> float:
    
    if has_direct_connection(curr_edge, goal_station, graph, time, TIME_DELTA) and has_changed_line:
        return float('inf')
    
    if number_of_changes == 0:
        return distance_fn(curr_station, goal_station) * DEGREES_TO_KILOMETERS_CONVERTER / AVERAGE_VELOCITY_IN_WROCLAW
    elif number_of_changes == 1:
        return distance_fn(curr_station, goal_station) * DEGREES_TO_KILOMETERS_CONVERTER / AVERAGE_VELOCITY_IN_WROCLAW * 2.25

    return distance_fn(curr_station, goal_station) * DEGREES_TO_KILOMETERS_CONVERTER / AVERAGE_VELOCITY_IN_WROCLAW  * (number_of_changes ** 2)

Tutaj sytuacja jest podobna - `distance_fn` to funkcja obliczająca odległość pomiędzy dwoma podanymi przystankami.

<h3>Porównanie rezultatów dla poszczególnych algorytmów cz.1<h3>
<h4>Trasa: Kwiska - pl. Grunwaldzki<h4>
<h4>Godzina startowa: 9:00<h4>

In [10]:
start_station = kwiska
end_station = pl_grunwaldzki
start_hour = "09:00:00"

exercise_1a(start_station, end_station, start_hour)

Dijkstra algorithm duration time: 0.7423 s
You need 31 minutes to get from Kwiska to PL. GRUNWALDZKI.
YOUR TIME: 09:00:00
LINE |   TIME   | STATION
-----+----------+---------------------------------
 20  | 09:01:00 | Kwiska
 20  | 09:02:00 | Małopanewska
 20  | 09:04:00 | Niedźwiedzia
 20  | 09:06:00 | Wrocław Mikołajów (Zachodnia)
----------------------------------------
 22  | 09:08:00 | Wrocław Mikołajów (Zachodnia)
 22  | 09:10:00 | pl. Strzegomski (Muzeum Współczesne)
 22  | 09:11:00 | Młodych Techników
 22  | 09:13:00 | PL. JANA PAWŁA II
----------------------------------------
 10  | 09:16:00 | PL. JANA PAWŁA II
 10  | 09:18:00 | Rynek
 10  | 09:19:00 | Zamkowa
 10  | 09:21:00 | Świdnicka
 10  | 09:23:00 | GALERIA DOMINIKAŃSKA
 10  | 09:26:00 | Urząd Wojewódzki (Muzeum Narodowe)
 10  | 09:27:00 | Katedra
 10  | 09:28:00 | Reja
 10  | 09:31:00 | PL. GRUNWALDZKI


In [11]:
exercise_1b(start_station, end_station, start_hour)

A* algorithm (time) duration time: 0.3260 s
You need 31 minutes to get from Kwiska to PL. GRUNWALDZKI.
YOUR TIME: 09:00:00
LINE |   TIME   | STATION
-----+----------+---------------------------------
 20  | 09:01:00 | Kwiska
 20  | 09:02:00 | Małopanewska
 20  | 09:04:00 | Niedźwiedzia
 20  | 09:06:00 | Wrocław Mikołajów (Zachodnia)
----------------------------------------
 22  | 09:08:00 | Wrocław Mikołajów (Zachodnia)
 22  | 09:10:00 | pl. Strzegomski (Muzeum Współczesne)
 22  | 09:11:00 | Młodych Techników
 22  | 09:13:00 | PL. JANA PAWŁA II
----------------------------------------
 10  | 09:16:00 | PL. JANA PAWŁA II
 10  | 09:18:00 | Rynek
 10  | 09:19:00 | Zamkowa
 10  | 09:21:00 | Świdnicka
 10  | 09:23:00 | GALERIA DOMINIKAŃSKA
 10  | 09:26:00 | Urząd Wojewódzki (Muzeum Narodowe)
 10  | 09:27:00 | Katedra
 10  | 09:28:00 | Reja
 10  | 09:31:00 | PL. GRUNWALDZKI


In [12]:
exercise_1c(start_station, end_station, start_hour)

A* algorithm (transfers) duration time: 0.3482 s
You need 0 transfers to get from Kwiska to PL. GRUNWALDZKI.
YOUR TIME: 09:00:00
LINE |   TIME   | STATION
-----+----------+---------------------------------
 10  | 09:07:00 | Kwiska
 10  | 09:08:00 | Małopanewska
 10  | 09:10:00 | Niedźwiedzia
 10  | 09:12:00 | Wrocław Mikołajów (Zachodnia)
 10  | 09:14:00 | pl. Strzegomski (Muzeum Współczesne)
 10  | 09:15:00 | Młodych Techników
 10  | 09:17:00 | PL. JANA PAWŁA II
 10  | 09:20:00 | Rynek
 10  | 09:21:00 | Zamkowa
 10  | 09:23:00 | Świdnicka
 10  | 09:25:00 | GALERIA DOMINIKAŃSKA
 10  | 09:28:00 | Urząd Wojewódzki (Muzeum Narodowe)
 10  | 09:29:00 | Katedra
 10  | 09:31:00 | Reja
 10  | 09:34:00 | PL. GRUNWALDZKI


<h3>Porównanie rezultatów dla poszczególnych algorytmów cz.2<h3>
<h4>Trasa: pl. Daniłowskiego - pl. Grunwaldzki<h4>
<h4>Godzina startowa: 05:40:00<h4>

In [13]:
start_station = pl_danilowskiego
end_station = pl_grunwaldzki
start_hour = "05:40:00"

exercise_1a(start_station, end_station, start_hour)

Dijkstra algorithm duration time: 0.6776 s
You need 21 minutes to get from pl. Daniłowskiego to PL. GRUNWALDZKI.
YOUR TIME: 05:40:00
LINE |   TIME   | STATION
-----+----------+---------------------------------
130  | 05:43:00 | pl. Daniłowskiego
130  | 05:44:00 | Berenta
130  | 05:47:00 | KROMERA
130  | 05:48:00 | Kromera (Czajkowskiego)
130  | 05:49:00 | Grudziądzka
130  | 05:50:00 | Brücknera
----------------------------------------
  D  | 05:51:00 | Brücknera
  D  | 05:53:00 | Kwidzyńska
  D  | 05:54:00 | Zacisze
  D  | 05:55:00 | Śniadeckich
  D  | 05:57:00 | Kochanowskiego
  D  | 06:01:00 | PL. GRUNWALDZKI


In [14]:
exercise_1b(start_station, end_station, start_hour)

A* algorithm (time) duration time: 0.0899 s
You need 21 minutes to get from pl. Daniłowskiego to PL. GRUNWALDZKI.
YOUR TIME: 05:40:00
LINE |   TIME   | STATION
-----+----------+---------------------------------
130  | 05:43:00 | pl. Daniłowskiego
130  | 05:44:00 | Berenta
130  | 05:47:00 | KROMERA
130  | 05:48:00 | Kromera (Czajkowskiego)
130  | 05:49:00 | Grudziądzka
130  | 05:50:00 | Brücknera
----------------------------------------
  D  | 05:51:00 | Brücknera
  D  | 05:53:00 | Kwidzyńska
  D  | 05:54:00 | Zacisze
  D  | 05:55:00 | Śniadeckich
  D  | 05:57:00 | Kochanowskiego
  D  | 06:01:00 | PL. GRUNWALDZKI


In [15]:
exercise_1c(start_station, end_station, start_hour)

A* algorithm (transfers) duration time: 0.1509 s
You need 0 transfers to get from pl. Daniłowskiego to PL. GRUNWALDZKI.
YOUR TIME: 05:40:00
LINE |   TIME   | STATION
-----+----------+---------------------------------
116  | 05:54:00 | pl. Daniłowskiego
116  | 05:55:00 | Kasprowicza
116  | 05:56:00 | Berenta
116  | 05:59:00 | KROMERA
116  | 06:01:00 | Kwidzyńska
116  | 06:02:00 | Zacisze
116  | 06:03:00 | Śniadeckich
116  | 06:05:00 | Kochanowskiego
116  | 06:08:00 | PL. GRUNWALDZKI


<h3>Porównanie rezultatów dla poszczególnych algorytmów cz.3<h3>
<h4>Trasa: KOSZAROWA (Uniwersytet) - GRABISZYŃSKA (Cmentarz)<h4>
<h4>Godzina startowa: 21:00:00<h4>

In [16]:
start_station = koszarowa_uniwersytet
end_station = grabiszynska_cmentarz
start_hour = "21:00:00"

exercise_1a(start_station, end_station, start_hour)

Dijkstra algorithm duration time: 0.6789 s
You need 46 minutes to get from KOSZAROWA (Uniwersytet) to GRABISZYŃSKA (Cmentarz).
YOUR TIME: 21:00:00
LINE |   TIME   | STATION
-----+----------+---------------------------------
  A  | 21:07:00 | KOSZAROWA (Uniwersytet)
  A  | 21:08:00 | KOSZAROWA (Szpital)
----------------------------------------
116  | 21:09:00 | KOSZAROWA (Szpital)
116  | 21:11:00 | pl. Daniłowskiego
----------------------------------------
  A  | 21:12:00 | pl. Daniłowskiego
  A  | 21:13:00 | Berenta
  A  | 21:16:00 | KROMERA
  A  | 21:18:00 | Mosty Warszawskie
  A  | 21:20:00 | Wyszyńskiego
  A  | 21:22:00 | Ogród Botaniczny
  A  | 21:23:00 | Katedra
  A  | 21:24:00 | Urząd Wojewódzki (Muzeum Narodowe)
  A  | 21:25:00 | Poczta Główna
  A  | 21:27:00 | skwer Krasińskiego
  A  | 21:28:00 | Wzgórze Partyzantów
  A  | 21:30:00 | Renoma
  A  | 21:32:00 | Arkady (Capitol)
  A  | 21:36:00 | Pl. Hirszfelda
  A  | 21:38:00 | Krucza
  A  | 21:39:00 | Krucza (Mielecka)
  A  | 21:

In [17]:
exercise_1b(start_station, end_station, start_hour)

A* algorithm (time) duration time: 0.1905 s
You need 46 minutes to get from KOSZAROWA (Uniwersytet) to GRABISZYŃSKA (Cmentarz).
YOUR TIME: 21:00:00
LINE |   TIME   | STATION
-----+----------+---------------------------------
  A  | 21:07:00 | KOSZAROWA (Uniwersytet)
  A  | 21:08:00 | KOSZAROWA (Szpital)
----------------------------------------
116  | 21:09:00 | KOSZAROWA (Szpital)
116  | 21:11:00 | pl. Daniłowskiego
----------------------------------------
  A  | 21:12:00 | pl. Daniłowskiego
  A  | 21:13:00 | Berenta
  A  | 21:16:00 | KROMERA
  A  | 21:18:00 | Mosty Warszawskie
  A  | 21:20:00 | Wyszyńskiego
  A  | 21:22:00 | Ogród Botaniczny
  A  | 21:23:00 | Katedra
  A  | 21:24:00 | Urząd Wojewódzki (Muzeum Narodowe)
  A  | 21:25:00 | Poczta Główna
  A  | 21:27:00 | skwer Krasińskiego
  A  | 21:28:00 | Wzgórze Partyzantów
  A  | 21:30:00 | Renoma
  A  | 21:32:00 | Arkady (Capitol)
  A  | 21:36:00 | Pl. Hirszfelda
  A  | 21:38:00 | Krucza
  A  | 21:39:00 | Krucza (Mielecka)
  A  | 21

In [18]:
exercise_1c(start_station, end_station, start_hour)

A* algorithm (transfers) duration time: 0.1375 s
You need 0 transfers to get from KOSZAROWA (Uniwersytet) to GRABISZYŃSKA (Cmentarz).
YOUR TIME: 21:00:00
LINE |   TIME   | STATION
-----+----------+---------------------------------
  A  | 21:07:00 | KOSZAROWA (Uniwersytet)
  A  | 21:08:00 | KOSZAROWA (Szpital)
  A  | 21:12:00 | pl. Daniłowskiego
  A  | 21:13:00 | Berenta
  A  | 21:16:00 | KROMERA
  A  | 21:18:00 | Mosty Warszawskie
  A  | 21:20:00 | Wyszyńskiego
  A  | 21:22:00 | Ogród Botaniczny
  A  | 21:23:00 | Katedra
  A  | 21:24:00 | Urząd Wojewódzki (Muzeum Narodowe)
  A  | 21:25:00 | Poczta Główna
  A  | 21:27:00 | skwer Krasińskiego
  A  | 21:28:00 | Wzgórze Partyzantów
  A  | 21:30:00 | Renoma
  A  | 21:32:00 | Arkady (Capitol)
  A  | 21:36:00 | Pl. Hirszfelda
  A  | 21:38:00 | Krucza
  A  | 21:39:00 | Krucza (Mielecka)
  A  | 21:41:00 | Inżynierska
  A  | 21:43:00 | Aleja Pracy
  A  | 21:45:00 | FAT
  A  | 21:46:00 | GRABISZYŃSKA (Cmentarz)


<h3>Porównanie średnich oraz min./max. czasów dla poszczególnych algorytmów<h3>

In [19]:
my_timer = Timer()
dijkstra_avg_time, dijkstra_min_time, dijkstra_max_time = my_timer.get_stats_time(10, exercise_1a, start_station, end_station, start_hour, False)
astar_time_avg_time, astar_time_min_time,astar_time_max_time = my_timer.get_stats_time(10, exercise_1c, start_station, end_station, start_hour, False)
astar_transfers_avg_time, astar_transfers_min_time, astar_transfers_max_time = my_timer.get_stats_time(10, exercise_1c, start_station, end_station, start_hour, False)

print(f"Dijkstra average time: {dijkstra_avg_time} s")
print(f"Dijkstra min and max time: {dijkstra_min_time}/{dijkstra_max_time} s\n")

print(f"A* (time) average time: {astar_time_avg_time} s")
print(f"A* (time) min and max time: {astar_time_min_time}/{astar_time_max_time} s\n")

print(f"A* (transfers) average time: {astar_transfers_avg_time} s")
print(f"A* (transfers) min and max time: {astar_transfers_min_time}/{astar_transfers_max_time} s")

Dijkstra average time: 0.7068644762039185 s
Dijkstra min and max time: 0.6565959453582764/0.7835137844085693 s

A* (time) average time: 0.1536926031112671 s
A* (time) min and max time: 0.14257287979125977/0.166304349899292 s

A* (transfers) average time: 0.1397191047668457 s
A* (transfers) min and max time: 0.13191986083984375/0.14806818962097168 s


<h3>Porównanie czasów dla różnych funkcji obliczającej dystans (Euclidean, Manhattan)<h3>

In [20]:
astar_time_euclidean_avg_time, astar_time_euclidean_min_time, astar_time_euclidean_max_time = my_timer.get_stats_time(10, exercise_1c, start_station, end_station, start_hour, False, False, euclidean_distance_stations)
astar_time_manhattan_avg_time, astar_time_manhattan_min_time, astar_time_manhattan_max_time = my_timer.get_stats_time(10, exercise_1c, start_station, end_station, start_hour, False, False, manhattan_distance_stations)


astar_transfers_euclidean_avg_time, astar_transfers_euclidean_min_time, astar_transfers_euclidean_max_time = my_timer.get_stats_time(10, exercise_1c, start_station, end_station, start_hour, False, False, euclidean_distance_stations)
astar_transfers_manhattan_avg_time, astar_transfers_manhattan_min_time, astar_transfers_manhattan_max_time = my_timer.get_stats_time(10, exercise_1c, start_station, end_station, start_hour, False, False, manhattan_distance_stations)

print("A* z kryterium czasowym")
print(f"Euclidean average time: {astar_time_euclidean_avg_time} s")
print(f"Euclidean min/max time: {astar_time_euclidean_min_time}/{astar_time_euclidean_max_time} s")


print(f"Manhattan average time: {astar_time_manhattan_avg_time} s")
print(f"Manhattan min/max time: {astar_time_manhattan_min_time}/{astar_time_manhattan_max_time} s")


print("\nA* z kryterium przesiadek")
print(f"Euclidean average time: {astar_transfers_euclidean_avg_time} s")
print(f"Euclidean min/max time: {astar_transfers_euclidean_min_time}/{astar_transfers_euclidean_max_time} s")


print(f"Manhattan average time: {astar_transfers_manhattan_avg_time} s")
print(f"Manhattan min/max time: {astar_transfers_manhattan_min_time}/{astar_transfers_manhattan_max_time} s")



A* z kryterium czasowym
Euclidean average time: 0.15572044849395753 s
Euclidean min/max time: 0.13040852546691895/0.17759966850280762 s
Manhattan average time: 0.16872291564941405 s
Manhattan min/max time: 0.15711236000061035/0.19431233406066895 s

A* z kryterium przesiadek
Euclidean average time: 0.13644764423370362 s
Euclidean min/max time: 0.12964701652526855/0.13962173461914062 s
Manhattan average time: 0.14396347999572753 s
Manhattan min/max time: 0.12651610374450684/0.17034173011779785 s


<h3>Porównanie tras dla zastosowanej funkcji dystansu w funkcji heurystycznej<h3>

In [21]:
print("A* - kryterium czasowe")
print("Euclidean:")
exercise_1b(start_station, end_station, start_hour, True, False, euclidean_distance_stations)

print("\nManhattan:")
exercise_1b(start_station, end_station, start_hour, True, False, manhattan_distance_stations)

A* - kryterium czasowe
Euclidean:
A* algorithm (time) duration time: 0.2234 s
You need 46 minutes to get from KOSZAROWA (Uniwersytet) to GRABISZYŃSKA (Cmentarz).
YOUR TIME: 21:00:00
LINE |   TIME   | STATION
-----+----------+---------------------------------
  A  | 21:07:00 | KOSZAROWA (Uniwersytet)
  A  | 21:08:00 | KOSZAROWA (Szpital)
----------------------------------------
116  | 21:09:00 | KOSZAROWA (Szpital)
116  | 21:11:00 | pl. Daniłowskiego
----------------------------------------
  A  | 21:12:00 | pl. Daniłowskiego
  A  | 21:13:00 | Berenta
  A  | 21:16:00 | KROMERA
  A  | 21:18:00 | Mosty Warszawskie
  A  | 21:20:00 | Wyszyńskiego
  A  | 21:22:00 | Ogród Botaniczny
  A  | 21:23:00 | Katedra
  A  | 21:24:00 | Urząd Wojewódzki (Muzeum Narodowe)
  A  | 21:25:00 | Poczta Główna
  A  | 21:27:00 | skwer Krasińskiego
  A  | 21:28:00 | Wzgórze Partyzantów
  A  | 21:30:00 | Renoma
  A  | 21:32:00 | Arkady (Capitol)
  A  | 21:36:00 | Pl. Hirszfelda
  A  | 21:38:00 | Krucza
  A  | 21:3

In [22]:
print("A* - kryterium przesiadek")
print("Euclidean:")
exercise_1c(start_station, end_station, start_hour, True, False, euclidean_distance_stations)

print("\nManhattan:")
exercise_1c(start_station, end_station, start_hour, True, False, manhattan_distance_stations)

A* - kryterium przesiadek
Euclidean:
A* algorithm (transfers) duration time: 0.1776 s
You need 0 transfers to get from KOSZAROWA (Uniwersytet) to GRABISZYŃSKA (Cmentarz).
YOUR TIME: 21:00:00
LINE |   TIME   | STATION
-----+----------+---------------------------------
  A  | 21:07:00 | KOSZAROWA (Uniwersytet)
  A  | 21:08:00 | KOSZAROWA (Szpital)
  A  | 21:12:00 | pl. Daniłowskiego
  A  | 21:13:00 | Berenta
  A  | 21:16:00 | KROMERA
  A  | 21:18:00 | Mosty Warszawskie
  A  | 21:20:00 | Wyszyńskiego
  A  | 21:22:00 | Ogród Botaniczny
  A  | 21:23:00 | Katedra
  A  | 21:24:00 | Urząd Wojewódzki (Muzeum Narodowe)
  A  | 21:25:00 | Poczta Główna
  A  | 21:27:00 | skwer Krasińskiego
  A  | 21:28:00 | Wzgórze Partyzantów
  A  | 21:30:00 | Renoma
  A  | 21:32:00 | Arkady (Capitol)
  A  | 21:36:00 | Pl. Hirszfelda
  A  | 21:38:00 | Krucza
  A  | 21:39:00 | Krucza (Mielecka)
  A  | 21:41:00 | Inżynierska
  A  | 21:43:00 | Aleja Pracy
  A  | 21:45:00 | FAT
  A  | 21:46:00 | GRABISZYŃSKA (Cmentarz

In [23]:
start_station = pl_danilowskiego
end_station = pl_grunwaldzki
start_hour = "05:40:00"

print("A* - kryterium czasowe")
print("Euclidean:")
exercise_1b(start_station, end_station, start_hour, True, False, euclidean_distance_stations)

print("\nManhattan:")
exercise_1b(start_station, end_station, start_hour, True, False, manhattan_distance_stations)

A* - kryterium czasowe
Euclidean:
A* algorithm (time) duration time: 0.0914 s
You need 21 minutes to get from pl. Daniłowskiego to PL. GRUNWALDZKI.
YOUR TIME: 05:40:00
LINE |   TIME   | STATION
-----+----------+---------------------------------
130  | 05:43:00 | pl. Daniłowskiego
130  | 05:44:00 | Berenta
130  | 05:47:00 | KROMERA
130  | 05:48:00 | Kromera (Czajkowskiego)
130  | 05:49:00 | Grudziądzka
130  | 05:50:00 | Brücknera
----------------------------------------
  D  | 05:51:00 | Brücknera
  D  | 05:53:00 | Kwidzyńska
  D  | 05:54:00 | Zacisze
  D  | 05:55:00 | Śniadeckich
  D  | 05:57:00 | Kochanowskiego
  D  | 06:01:00 | PL. GRUNWALDZKI

Manhattan:
A* algorithm (time) duration time: 0.0922 s
You need 21 minutes to get from pl. Daniłowskiego to PL. GRUNWALDZKI.
YOUR TIME: 05:40:00
LINE |   TIME   | STATION
-----+----------+---------------------------------
130  | 05:43:00 | pl. Daniłowskiego
130  | 05:44:00 | Berenta
130  | 05:47:00 | KROMERA
130  | 05:48:00 | Kromera (Czajkows

In [24]:
print("A* - kryterium przesiadek")
print("Euclidean:")
exercise_1c(start_station, end_station, start_hour, True, False, euclidean_distance_stations)

print("\nManhattan:")
exercise_1c(start_station, end_station, start_hour, True, False, manhattan_distance_stations)

A* - kryterium przesiadek
Euclidean:
A* algorithm (transfers) duration time: 0.1790 s
You need 0 transfers to get from pl. Daniłowskiego to PL. GRUNWALDZKI.
YOUR TIME: 05:40:00
LINE |   TIME   | STATION
-----+----------+---------------------------------
116  | 05:54:00 | pl. Daniłowskiego
116  | 05:55:00 | Kasprowicza
116  | 05:56:00 | Berenta
116  | 05:59:00 | KROMERA
116  | 06:01:00 | Kwidzyńska
116  | 06:02:00 | Zacisze
116  | 06:03:00 | Śniadeckich
116  | 06:05:00 | Kochanowskiego
116  | 06:08:00 | PL. GRUNWALDZKI

Manhattan:
A* algorithm (transfers) duration time: 0.1459 s
You need 0 transfers to get from pl. Daniłowskiego to PL. GRUNWALDZKI.
YOUR TIME: 05:40:00
LINE |   TIME   | STATION
-----+----------+---------------------------------
116  | 05:54:00 | pl. Daniłowskiego
116  | 05:55:00 | Kasprowicza
116  | 05:56:00 | Berenta
116  | 05:59:00 | KROMERA
116  | 06:01:00 | Kwidzyńska
116  | 06:02:00 | Zacisze
116  | 06:03:00 | Śniadeckich
116  | 06:05:00 | Kochanowskiego
116  | 06:0

In [25]:
start_station = kwiska
end_station = pl_grunwaldzki
start_hour = "09:00:00"

print("A* - kryterium czasowe")
print("Euclidean:")
exercise_1b(start_station, end_station, start_hour, True, False, euclidean_distance_stations)

print("\nManhattan:")
exercise_1b(start_station, end_station, start_hour, True, False, manhattan_distance_stations)

A* - kryterium czasowe
Euclidean:
A* algorithm (time) duration time: 0.3102 s
You need 31 minutes to get from Kwiska to PL. GRUNWALDZKI.
YOUR TIME: 09:00:00
LINE |   TIME   | STATION
-----+----------+---------------------------------
 20  | 09:01:00 | Kwiska
 20  | 09:02:00 | Małopanewska
 20  | 09:04:00 | Niedźwiedzia
 20  | 09:06:00 | Wrocław Mikołajów (Zachodnia)
----------------------------------------
 22  | 09:08:00 | Wrocław Mikołajów (Zachodnia)
 22  | 09:10:00 | pl. Strzegomski (Muzeum Współczesne)
 22  | 09:11:00 | Młodych Techników
 22  | 09:13:00 | PL. JANA PAWŁA II
----------------------------------------
 10  | 09:16:00 | PL. JANA PAWŁA II
 10  | 09:18:00 | Rynek
 10  | 09:19:00 | Zamkowa
 10  | 09:21:00 | Świdnicka
 10  | 09:23:00 | GALERIA DOMINIKAŃSKA
 10  | 09:26:00 | Urząd Wojewódzki (Muzeum Narodowe)
 10  | 09:27:00 | Katedra
 10  | 09:28:00 | Reja
 10  | 09:31:00 | PL. GRUNWALDZKI

Manhattan:
A* algorithm (time) duration time: 0.2861 s
You need 31 minutes to get fr

In [26]:
print("A* - kryterium przesiadek")
print("Euclidean:")
exercise_1c(start_station, end_station, start_hour, True, False, euclidean_distance_stations)

print("\nManhattan:")
exercise_1c(start_station, end_station, start_hour, True, False, manhattan_distance_stations)

A* - kryterium przesiadek
Euclidean:
A* algorithm (transfers) duration time: 0.2936 s
You need 0 transfers to get from Kwiska to PL. GRUNWALDZKI.
YOUR TIME: 09:00:00
LINE |   TIME   | STATION
-----+----------+---------------------------------
 10  | 09:07:00 | Kwiska
 10  | 09:08:00 | Małopanewska
 10  | 09:10:00 | Niedźwiedzia
 10  | 09:12:00 | Wrocław Mikołajów (Zachodnia)
 10  | 09:14:00 | pl. Strzegomski (Muzeum Współczesne)
 10  | 09:15:00 | Młodych Techników
 10  | 09:17:00 | PL. JANA PAWŁA II
 10  | 09:20:00 | Rynek
 10  | 09:21:00 | Zamkowa
 10  | 09:23:00 | Świdnicka
 10  | 09:25:00 | GALERIA DOMINIKAŃSKA
 10  | 09:28:00 | Urząd Wojewódzki (Muzeum Narodowe)
 10  | 09:29:00 | Katedra
 10  | 09:31:00 | Reja
 10  | 09:34:00 | PL. GRUNWALDZKI

Manhattan:
A* algorithm (transfers) duration time: 0.3791 s
You need 0 transfers to get from Kwiska to PL. GRUNWALDZKI.
YOUR TIME: 09:00:00
LINE |   TIME   | STATION
-----+----------+---------------------------------
 10  | 09:07:00 | Kwiska

<h3>Kod dla algorytmu Tabu search<h3>

In [27]:
def tabu_search(start_station, stations_to_visit, start_time, graph, to_print: bool = False, heuristic_fn = manhattan_distance_stations):
    aspiration_criteria = 100
    if start_station == None or stations_to_visit == []:
        return
    turns_improved = 0
    improve_thresh = 2 * math.floor(math.sqrt(MAX_ITERATIONS))
    tabu_list = []
    tabu_tenure = len(stations_to_visit)
    current_solution = [start_station] + stations_to_visit + [start_station]
    best_solution = current_solution[:]
    best_solution_cost, best_solution_edges = get_solution_cost(best_solution, start_time, graph, heuristic_fn)

    for iteration in range(MAX_ITERATIONS):
        if turns_improved > improve_thresh:
            break
        best_neighbor = None
        best_neighbor_cost = float('inf')
        best_neighbor_edges = []
        tabu_candidate = (0,0)
        for i in range(1, len(stations_to_visit) + 1):
            for j in range(i+1, len(stations_to_visit) + 1):
                neighbor = current_solution[:]
                neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
                neighbor_cost, neighbor_edges = get_solution_cost(neighbor, start_time, graph, heuristic_fn)
                if (i,j) not in tabu_list or neighbor_cost < aspiration_criteria:
                    if neighbor_cost < best_neighbor_cost:
                        best_neighbor = neighbor[:]
                        best_neighbor_cost = neighbor_cost
                        best_neighbor_edges = neighbor_edges
                        tabu_candidate = (i,j)
        if best_neighbor is not None:
            current_solution = best_neighbor[:]
            tabu_list.append(tabu_candidate)

            if len(tabu_list) > tabu_tenure:
                tabu_list.pop(0)

            if best_neighbor_cost < best_solution_cost:
                best_solution = best_neighbor[:]
                best_solution_cost = best_neighbor_cost
                best_solution_edges = best_neighbor_edges
                turns_improved = 0
            else:
                turns_improved = turns_improved + 1

        if to_print: print("Iteration {}: Best solution cost = {}".format(iteration, best_solution_cost))

    if to_print:
        print_best_solution(best_solution)
        print(f"Best solution cost: {best_solution_cost} minutes")
        print(f"Start time: {start_time}")
        print_formatted_edges(start_station, best_solution_edges)

    return best_neighbor_cost, best_solution, best_neighbor_edges

<h3>Rezultat dla algorytmu Tabu search<h3>

In [28]:
hour = '21:05:00'
start_station = koszarowa_uniwersytet
stations_to_visit = [
    fat,
    wiejska,
    poczta_glowna,
    krzyki
]
_ = tabu_search(start_station, stations_to_visit, hour, data, True)

Iteration 0: Best solution cost = 126
Iteration 1: Best solution cost = 126
Iteration 2: Best solution cost = 126
Iteration 3: Best solution cost = 126
Iteration 4: Best solution cost = 126
Iteration 5: Best solution cost = 126
Iteration 6: Best solution cost = 126
Iteration 7: Best solution cost = 126
Best solution: [KOSZAROWA (Uniwersytet) - Poczta Główna - Wiejska - FAT - KRZYKI - KOSZAROWA (Uniwersytet)]
Best solution cost: 126 minutes
Start time: 21:05:00
--------------------------------------------------
KOSZAROWA (Uniwersytet)                  |       | 21:09:00
pl. Daniłowskiego                        | 116   | 21:11:00
--------------------------------------------------
pl. Daniłowskiego                        | A     | 21:12:00
Berenta                                  | A     | 21:13:00
KROMERA                                  | A     | 21:16:00
Mosty Warszawskie                        | A     | 21:18:00
Wyszyńskiego                             | A     | 21:20:00
Ogród Botanic